# Extension of Albu to RGB Only
So far unsuccessful attempt to make the model work with only 3 channels again. Currently in progress.

In [1]:
save_mem = True
if not save_mem:
    # When we only have satellite imagery
    rgb_model = read_model(cfg, 3)

    sd = rgb_model.state_dict()
    skip_layers = rgb_model.first_layer_params_names
    print('skipping: ', [k for k in pretrained_dict.keys() if any(s in k for s in skip_layers)])
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if not any(s in k for s in skip_layers)}
    sd.update(pretrained_dict)
    rgb_model.load_state_dict(sd)

    output3 = predict_on_file(rgb_model, fname, crop=True, channels=3)
    print("F1 / IOU with crop + flip:", 
          score_pred(thresh(output3),lbl))
    print("F1 / IOU with no crop + flip:", 
          score_pred(thresh(predict_on_file(rgb_model, fname, crop=False, channels=3)), lbl))
    print("F1 / IOU with no crop + no flip:",
         score_pred(thresh(predict_on_file(rgb_model, fname, crop=False, channels=3, flips=flip.FLIP_NONE)), lbl))